In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL"
os.chdir(path)
os.listdir(path)
import os
print(os.getcwd())

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL


In [2]:
!pip install accelerate
!pip install transformers
!pip install datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
torch.autograd.set_detect_anomaly(True)



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2Model
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

def load_data(data_path):
    try:
        data_frame = pd.read_csv(data_path, sep='|', header=None, names=[
            "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
        ])
    except UnicodeDecodeError:
        data_frame = pd.read_csv(data_path, sep='|', header=None, names=[
            "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
        ], encoding='ISO-8859-1')
    return data_frame

def read_corpus(data_frame, tokenizer, max_length=128, word_sense_dict=None):
    data = []
    is_dict_provided = word_sense_dict is not None
    word_sense_dict = word_sense_dict or {}

    data_frame = data_frame.fillna('').astype(str)

    for index, item in data_frame.iterrows():
        word = item['Abbreviation']
        sense = item['Long_form']
        if not is_dict_provided:
            word_sense_dict.setdefault(word, set()).add(sense)

        sentence = item['Context']
        pos_input = tokenizer.encode_plus(sentence + ' [SEP] ' + word + ' [SEP] ' + sense,
                                          padding='max_length', max_length=max_length,
                                          truncation=True, return_tensors='pt')
        data.append((pos_input['input_ids'], pos_input['attention_mask'], 1))

        if word not in word_sense_dict:
            word_sense_dict[word] = set()
        word_sense_dict[word].add(sense)

        for word_sense in word_sense_dict[word]:
            if word_sense != sense:
                neg_input = tokenizer.encode_plus(sentence + ' [SEP] ' + word + ' [SEP] ' + word_sense,
                                                  padding='max_length', max_length=max_length,
                                                  truncation=True, return_tensors='pt')
                data.append((neg_input['input_ids'], neg_input['attention_mask'], 0))

    return data, word_sense_dict




class WSDDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = torch.tensor(item[0], dtype=torch.long)
        attention_mask = torch.tensor(item[1], dtype=torch.long)
        labels = torch.tensor(item[2], dtype=torch.long)  # 确保标签是一个标量，形状为 [1]
        return input_ids, attention_mask, labels


# Example usage
data_path = "AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt"
data_frame = load_data(data_path)
train_df, valid_df = train_test_split(data_frame, test_size=0.2, random_state=42)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token


train_data, train_word_sense_dict = read_corpus(train_df, tokenizer)
valid_data, _ = read_corpus(valid_df, tokenizer, word_sense_dict=train_word_sense_dict)

train_dataset = WSDDataset(train_data)
valid_dataset = WSDDataset(valid_data)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
from transformers import GPT2PreTrainedModel, GPT2Model
import torch.nn as nn

class CustomGPT2ForWSD(GPT2PreTrainedModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.num_labels = num_labels
        self.gpt2 = GPT2Model(config)
        self.dropout = nn.Dropout(config.resid_pdrop)
        self.classifier = nn.Linear(config.n_embd, num_labels)
        self.post_init()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state  # shape: (batch_size, sequence_length, hidden_size)
        pooled_output = hidden_state[:, 0]  # Use the first token's representation
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)  # shape: (batch_size, num_labels)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits


from transformers import GPT2Config

# Configuration specific to GPT-2
config = GPT2Config.from_pretrained('gpt2')
config.hidden_dropout_prob = 0.1  # You can adjust dropout rate

# Instantiate the model
model = CustomGPT2ForWSD(config, num_labels=2)  # Ensure num_labels matches your task

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CustomGPT2ForWSD(
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
)

In [6]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from transformers import AdamW

# def collate_fn(batch):
#     input_ids = [item[0] for item in batch]
#     attention_mask = [item[1] for item in batch]
#     labels = [item[2] for item in batch]

#     input_ids = torch.stack(input_ids)
#     attention_mask = torch.stack(attention_mask)
#     labels = torch.tensor(labels)

#     return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# def collate_fn(batch):
#     input_ids = [item[0] for item in batch]
#     attention_mask = [item[1] for item in batch]
#     labels = [item[2] for item in batch]

#     input_ids = torch.stack(input_ids)
#     attention_mask = torch.stack(attention_mask)
#     labels = torch.tensor(labels)

#     return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def collate_fn(batch):
    input_ids = [item[0].squeeze(0) for item in batch]  # 去掉多余的维度
    attention_mask = [item[1].squeeze(0) for item in batch]  # 去掉多余的维度
    labels = [item[2].squeeze(0) for item in batch]  # 确保 labels 的形状为 [batch_size]

    # Convert lists to tensors
    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    labels = torch.stack(labels)  # 确保 labels 的形状为 [batch_size]

    # print(f"Collate function - input_ids shape: {input_ids.shape}")
    # print(f"Collate function - attention_mask shape: {attention_mask.shape}")
    # print(f"Collate function - labels shape: {labels.shape}")

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}





import torch
import numpy as np
from transformers import AdamW
from torch.utils.data import DataLoader



def train_model(model, train_loader, valid_loader, device, num_epochs=3):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for step, batch in enumerate(train_loader):
            input_ids, attention_mask, labels = (
                batch['input_ids'].to(device),
                batch['attention_mask'].to(device),
                batch['labels'].to(device)
            )
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, labels=labels)
            loss = outputs[0]

            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            if step % 100 == 0:
                print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.item()}")

        print(f"Epoch {epoch + 1}, Average Training Loss: {total_train_loss / len(train_loader)}")

        model.eval()
        total_eval_loss = 0
        total_eval_accuracy = 0
        total_eval_samples = 0
        all_preds = []
        all_labels = []
        contexts = []  # 用于保存验证样本的上下文

        for batch in valid_loader:
            input_ids, attention_mask, labels = (
                batch['input_ids'].to(device),
                batch['attention_mask'].to(device),
                batch['labels'].to(device)
            )
            with torch.no_grad():
                outputs = model(input_ids, attention_mask, labels=labels)
                loss, logits = outputs

            total_eval_loss += loss.item()
            preds = logits.argmax(dim=1)
            total_eval_accuracy += (preds == labels).sum().item()
            total_eval_samples += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            contexts.extend(input_ids.cpu().numpy())  # 保存输入的文本ID


        print(f"Epoch {epoch + 1}, Average Testing Loss: {total_eval_loss / len(valid_loader)}, Accuracy: {total_eval_accuracy / total_eval_samples}")

        # 打印验证样本的一些预测结果
        random_indices = np.random.choice(len(all_preds), 5, replace=False)
        print("Validation sample predictions:")
        for idx in random_indices:
            print(f"Sample ID: {idx}, Predicted: {all_preds[idx]}, True: {all_labels[idx]}")
            input_ids = contexts[idx]
            input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
            print(f"Context: {input_text}")


# 使用定义好的 collate_fn 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=100, shuffle=False, collate_fn=collate_fn)



def zero_shot_evaluation(model, valid_loader, device):
    model.eval()
    total_eval_loss = 0
    total_eval_accuracy = 0
    total_eval_samples = 0
    all_preds = []
    all_labels = []
    contexts = []  # 用于保存验证样本的上下文

    for batch in valid_loader:
        input_ids, attention_mask, labels = (
            batch['input_ids'].to(device),
            batch['attention_mask'].to(device),
            batch['labels'].to(device)
        )
        with torch.no_grad():
            outputs = model(input_ids, attention_mask, labels=labels)
            loss, logits = outputs

        total_eval_loss += loss.item()
        preds = logits.argmax(dim=1)
        total_eval_accuracy += (preds == labels).sum().item()
        total_eval_samples += labels.size(0)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        contexts.extend(input_ids.cpu().numpy())  # 保存输入的文本ID

    avg_loss = total_eval_loss / len(valid_loader)
    avg_accuracy = total_eval_accuracy / total_eval_samples
    print(f"Zero-Shot Evaluation - Average Loss: {avg_loss}, Accuracy: {avg_accuracy}")

    # 打印验证样本的一些预测结果
    random_indices = np.random.choice(len(all_preds), 5, replace=False)
    print("Validation sample predictions:")
    for idx in random_indices:
        print(f"Sample ID: {idx}, Predicted: {all_preds[idx]}, True: {all_labels[idx]}")
        input_ids = contexts[idx]
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
        print(f"Context: {input_text}")



# 进行 Zero-Shot 评估
zero_shot_evaluation(model, valid_loader, device)





<ipython-input-4-f6cc175a2bfb>:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(item[0], dtype=torch.long)
<ipython-input-4-f6cc175a2bfb>:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(item[1], dtype=torch.long)


Zero-Shot Evaluation - Average Loss: 1.2529959466559755, Accuracy: 0.2308425877503307
Validation sample predictions:
Sample ID: 9578, Predicted: 1, True: 0
Context: Now, she has no limp and denies any discomfort in the hip. She is really quite happy with the results. She was hospitalized in _%#MM#%_ of 1991 with a spontaneous AB and in _%#MM#%_ 1999, she had a term pregnancy which ended up with a cesarean section because of cephalopelvic disproportion secondary to the pelvic fractures and _______________ in the pelvis. [SEP] AB [SEP] arterial blood
Sample ID: 17308, Predicted: 1, True: 0
Context: 10. Depression. 11. Hypothyroidism. 12. Hypertension. MEDICATIONS: 1. Tylenol ES 500 mg p.o. each day at bedtime as needed for discomfort. 2. Zymar 0.3% one drop right eye 4x daily. 3. Pred-Forte 1% one drop right eye 4x daily. 4. Toprol XL 75 mg p.o. daily. [SEP] ES [SEP] enhanced sensitivity
Sample ID: 29384, Predicted: 1, True: 0
Context: The limitations of generalizing these reports to the

In [7]:
train_model(model, train_loader, valid_loader, device)  # 确保 loader 名称正确



<ipython-input-4-f6cc175a2bfb>:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(item[0], dtype=torch.long)
<ipython-input-4-f6cc175a2bfb>:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(item[1], dtype=torch.long)


Epoch 1, Step 0, Loss: 0.9914300441741943
Epoch 1, Step 100, Loss: 0.5877302289009094
Epoch 1, Step 200, Loss: 0.5948309898376465
Epoch 1, Step 300, Loss: 0.5515440106391907
Epoch 1, Step 400, Loss: 0.6001582741737366
Epoch 1, Step 500, Loss: 0.607919454574585
Epoch 1, Step 600, Loss: 0.6610959768295288
Epoch 1, Step 700, Loss: 0.5851086378097534
Epoch 1, Step 800, Loss: 0.6287432909011841
Epoch 1, Step 900, Loss: 0.6356479525566101
Epoch 1, Step 1000, Loss: 0.6737242341041565


KeyboardInterrupt: 